![](https://github.com/vorodrigues/databricks-automl-lab/blob/main/img/header-automl.png?raw=true)

# Lab 02 - Treinamento e Avaliação de Modelos

## Exercício 02.01 - Treinamento de modelos com AutoML

Após explorar e preparar nossos dados estamos prontos para começar a treinar nossos modelos usando o **AutoML**.

Para isso, vamos criar nosso experimento!

Siga os passos abaixo:

- No menu principal, selecione **Experiments**
- No cartão **Classification**, clique em **Start training**
- Em **Cluster**, selecione o seu cluster
- Em **Input training dataset**, clique em **Browse** e selecione a tabela **fraud_abt**
- Em **Prediction target**, selecione **fraud_report**
- Clique em **Advanced Configuration**
  - Em **Evaluation metric**, selecione **ROC/AUC**
  - Em **Experiment directory**, digite o caminho de um diretório compartilhado ao qual o grupo do cluster tenha acesso
  - Em **Timeout (minutes)**, digite **10**
  - Em **Positive label**, digite **1**
- Clique em **Start AutoML**

## Exercício 02.02 - Analisando o notebook de exploração

- Clique em **View data exploration notebook**
- Desça até **Profiling Results**
- Analise o **Profiling Report**
  - Use a sessão **Overview** para dados faltantes (missing), registros duplicados e alertas gerados
  - Use a sessão **Variables** para avaliar se as distribuições das variáveis estão normais
  - Use a tabela **Correlations** para identificar colunas com alta correlação

## Exercício 02.03 - Avaliando os modelos

- Clique no **ícone do gráfico**
- Clique no **ícone do olho** e selecione **Show all runs**
- Avalie a performance dos modelos
  - São criados automaticamente gráficos para diversas métricas de performance (F1, acuracidade, precisão, recall, entre outras) em cada uma das bases – treino (training), validação (val) e teste (test)
- No canto superior direito, clique em **Add chart** e selecione **Scatter plot**
  - Em **X axis**, selecione **training_roc_auc**
  - Em **Y axis**, selecione **test_roc_auc**
  - Clique em **Add chart**

Este gráfico nos permite comparar a performance dos modelos nas bases utilizadas durante os seus treinamentos e também em dados que o modelo não havia visto antes. Isso nos permite entender se o modelo está **generalizando** bem, ou seja, se ele é capaz de reproduzir sua performance do treinamento no mundo real. Vamos buscar os modelos que possuem a maior performance, porém com a menor diferença possível entre treino e teste.

## Exercício 02.04 - Visualizando o melhor modelo

- No gráfico, clique no ponto que representa o melhor modelo
- No cartão, clique no nome do modelo

Veja que todas as informações do modelo se encontram disponíveis, desde os parâmetros utilizados  e métricas de performance nas diversas bases, até o notebook criado pelo **AutoML** para treinar o modelo e artefatos gerados. Isso garante máxima transparência ao processo e permite com que vocês façam alterações adicionais, caso desejem.

Aficionalmente, siga os passos abaixo:

- Clique no notebook disponível em **Source** para visualizar o código
- Na tela do exeperimento, clique na aba **Artifacts** para visualizar as especificações do modelo, binários, dependências e outros artefatos necessários para a sua execução em produção

**Observação**:
- Alguns artefatos são gerados apenas para o modelo campeão

## Exercício 02.05 - Inferência

- Na célula abaixo, preencha o valor da variável `db`
- Na aba **Artifacts**, copie o código em **Predict on a Spark DataFrame** e cole na célula vazia na sequência
- Execute todas as células abaixo

In [0]:
df = spark.table('vr_demo.fraud.fraud_abt_sample')

In [0]:
import mlflow
from pyspark.sql.functions import struct, col
logged_model = 'runs:/21b4d7592a814a7ab6381f13c8054eaf/model'

# Load model as a Spark UDF. Override result_type if the model does not return double values.
loaded_model = mlflow.pyfunc.spark_udf(spark, model_uri=logged_model)

# Predict on a Spark DataFrame.
df = df.withColumn('predictions', loaded_model(struct(*map(col, df.columns))))

In [0]:
display(df)

**Observações:**

- O exemplo de inferência acima pode ser utilizado para fins de validação dos modelos gerados pelo AutoML, porém não é recomendada a sua utilização em produção.
- Para projetos produtivos, é recomendado:
  - Mover o experimento para um diretório seguro e ajustar as permissões para evitar alterações indesejadas
  - Registrar o modelo no **Unity Catalog**
  - Atribuir **aliases** para identificar modelos em validação e produção, por exemplo
  - Referenciar os modelos do **Unity Catalog** nos jobs de inferência

**Parabéns!**

Você concluiu o treinamento e avaliação de modelos. Agora você já possui os conceitos básicos para criar seus próprios modelos com agilidade, mas também aproveitando  boas práticas e técnicas de otimização embutidas no **Databricks AutoML**.